# December 13, 2023

https://adventofcode.com/2023/day/13

In [159]:
class MirrorMaze:
    def __init__( self, text ):
        self.text = text
        self.ncol = len(text[0])
        self.nrow = len(text)

        # indexed by the row above or col before
        self.__axis = None
        # row or col
        self.__axis_type = None

    def axis( self, smudged=False, reset=False ):
        if reset:
            self.__axis = None
            self.__axis_type = None

        if self.__axis is not None:
            return self.__axis, self.__axis_type
        
        i = 0

        # possible rows and columns
        prows = set( [r for r in range(self.nrow - 1)] )
        pcols = set( [c for c in range(self.ncol -1 )] )

        # possible rows and columns, if smudged
        psrows = set()
        pscols = set()

        while self.__axis is None:
            # print(i)
            # print(prows)
            # print(pcols)
            if i < self.ncol:
                for r in list(prows):
                    # determine how many rows we need to check for row r and col i
                    wid = min( r+1, self.nrow-(r+1) )
                    for iter in range( wid ):
                        #print("row:", r, iter)
                        # check the next element, remove the row it fails
                        if self.text[r-iter][i] != self.text[r+iter+1][i]:
                            if smudged:
                                if r in psrows:
                                    #print("rm from rows")
                                     # a second mistake... it's not just a smudge
                                    psrows.remove(r)
                                    prows.remove(r)
                                    break # we're done here. Check the next row in prows
                                else:
                                    #print("add smudge")
                                    # still a possibility if this error is a smudge
                                    psrows.add(r)
                                    #continue # this isn't doing anything besides emphasizing we're still going until a second difference
                            else:
                                #print("not", r)
                                prows.remove(r)
                                break

            if i < self.nrow:
                for c in list(pcols):
                    # determing how many cols we neec to check for row i and col c
                    wid = min( c+1, self.ncol-(c+1) )
                    for iter in range( wid ):
                        if self.text[i][ c-iter] != self.text[i][ c+iter+1 ]:
                            if smudged:
                                if c in pscols:
                                    # a second mistake... it's not just a smudge
                                    pscols.remove(c)
                                    pcols.remove(c)
                                    break # we're done here. Check the next row in prows
                                else:
                                    # still a possibility if this error is a smudge
                                    pscols.add(c)
                                    #continue # this isn't doing anything besides emphasizing we're still going until a second difference
                            else:
                                pcols.remove(c)
                                break

            # Done checking ith col/row
            # i = how many elements we've checked so far
            i += 1

            # Solution 1: we've validated a row
            if (i >= self.ncol) and (len(prows) >= 1):
                pr = prows.pop()
                # for smudge case, the row must also include the smudge
                # print("is it", pr)
                # print("prows", prows)
                # print("psrows", psrows)
                if (not smudged) or (pr in psrows):
                    # print("it is", pr, "!")
                    self.__axis = pr
                    self.__axis_type = "row"
                # but wait! there's another row that really is verified, smudge and all
                # (only one row can be completed sans smudge, so no need to check again)
                elif len(prows) >= 1:
                    self.__axis = prows.pop()
                    self.__axis_type = "row"
                else:
                    prows.clear()

            # Solution 2: we've validated a col
            elif (i >= self.nrow) and (len(pcols) >= 1):
                pc = pcols.pop()
                # for smudge case, the col must also include the smudge
                if (not smudged) or (pc in pscols):
                    self.__axis = pc
                    self.__axis_type = "col"
                # but wait! there's another col that really is verified, smudge and all
                # (only one col can be completed sans smudge, so no need to check again)
                elif len(pcols) >= 1:
                        self.__axis = pcols.pop()
                        self.__axis_type = "col"
                else:
                    pcols.clear()

            # Solution 3: There's one possible row and no possible cols
            elif (len(prows) == 1) and (len(pcols) == 0):
                self.__axis = prows.pop()
                self.__axis_type = "row"

            # Solution 4: There's one possible col and no possible rows
            elif (len(pcols) == 1) and (len(prows)==0):
                self.__axis = pcols.pop()
                self.__axis_type = "col"

        return self.__axis, self.__axis_type
    
    def __repr__( self ):
        return "\n".join(self.text)
    
    def __str__( self ):
        return "\n".join(self.text)

In [160]:
def parse_input( lines ):
    mazes = []
    text = []
    for line in lines:
        if len(line)==0:
            mazes.append( MirrorMaze(text) )
            text = []
        else:
            text.append( line )

    if len(text) > 0:
        mazes.append( MirrorMaze(text) )

    return mazes

In [161]:
test_str = f'''#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#'''
test = parse_input( test_str.split("\n") )

In [145]:
test[0]

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#

In [148]:
test[0].axis(smudged=True, reset=True)

0
{0, 1, 2, 3, 4, 5}
{0, 1, 2, 3, 4, 5, 6, 7}
row: 0 0
add smudge
row: 1 0
add smudge
row: 1 1
row: 2 0
row: 2 1
row: 2 2
add smudge
row: 3 0
add smudge
row: 3 1
rm from rows
row: 4 0
row: 4 1
row: 5 0
add smudge
1
{0, 1, 2, 4, 5}
{0, 1, 2, 4, 6, 7}
row: 0 0
row: 1 0
rm from rows
row: 2 0
row: 2 1
row: 2 2
row: 4 0
row: 4 1
add smudge
row: 5 0
2
{0, 2, 4, 5}
{0, 4}
row: 0 0
row: 2 0
row: 2 1
row: 2 2
row: 4 0
row: 4 1
rm from rows
row: 5 0
3
{0, 2, 5}
{0, 4}
row: 0 0
rm from rows
row: 2 0
row: 2 1
row: 2 2
row: 5 0
rm from rows
4
{2}
{0, 4}
row: 2 0
row: 2 1
row: 2 2
5
{2}
{0, 4}
row: 2 0
row: 2 1
row: 2 2
6
{2}
{0, 4}
row: 2 0
row: 2 1
row: 2 2
7
{2}
set()
row: 2 0
row: 2 1
row: 2 2


(2, 'row')

In [162]:
fn = "data/13.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = parse_input( [x.strip() for x in text] )

In [163]:
print( test[0] )

#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.


In [164]:
test[0].axis()

(4, 'col')

In [165]:
test[1].axis()

(3, 'row')

### Part 1

In [166]:
puzz[0].axis()

(1, 'col')

In [167]:
len(puzz)

100

In [168]:
puzz[9]

##..#..#..##..#
.###....###.#..
.#...##...#..#.
..##.##.##.....
.#.#.##.#.#.#.#
.....#........#
.#..####..#.##.
####.##.####.##
####.##.####.##

In [169]:
puzz[9].axis()

(7, 'row')

In [170]:
print( puzz[10] )
puzz[10].axis()

.#.#..####.
#.####.##.#
..#..######
..#...####.
..#...####.
..#..######
#.####.##.#
.#.#..####.
#..#.##..##
#.#.#######
###........
....##....#
..#.###..##
...#..#.##.
.#.#.##..##


(3, 'row')

In [171]:
# now solve all reflections
for p in puzz:
    p.axis()

In [173]:
part1 = 0
for p in puzz:
    i, d = p.axis()
    if d == "col":
        part1 += i+1
    else:
        part1 += 100*(i+1)
part1

32035

### Part 2

In [174]:
test[0].axis(smudged=True, reset=True)

(2, 'row')

In [175]:
test[1].axis(smudged=True, reset=True)

(0, 'row')

In [176]:
part2 = 0
for p in puzz:
    i, d = p.axis(smudged=True, reset=True)
    if d == "col":
        part2 += i+1
    else:
        part2 += 100*(i+1)
part2

24847